# 五. 决策树

决策树是一种基本的分类和回归方法。决策树模型呈树形结构，在分类问题中，表示基于特征对实例进行分类的过程，可认为是if-then规则的集合，也可认为是定义在特征空间与类空间上的条件概率分布。

决策树的有点在于可读性强，且分类速度快。学习时，利用训练数据，根据损失函数最小化的原则建立决策树模型。预测时，对新的数据，利用决策树模型进行分类。

决策树学习通常包含3个步骤：**特征选择、决策树的生成和决策树的修剪**。

决策树学习的思想主要源于`Quinlan(1986)`的`ID3`算法和`Quinlan(1993)`的`C4.5`算法，以及`Breiman等(1984)`提出的`CART`算法.

## 1. 决策树模型与学习

### 1.1 决策树模型

>定义5.1 （决策树）分类决策树模型是一种描述对实例进行分类的树形结构。决策树由结点(node)和有向边(directed edge)组成。结点又进一步分为内部结点(internal node)和叶结点(leaf node)。内部结点表示一个特征或属性，叶结点表示一个类。

用决策树分类，从根结点开始，对实例的某一特征进行测试，根据测试结果，将实例分配到其子节点；这时，每一个子节点对应着该特征的一个取值。如此递归地对实例进行测试并分配，直至达到叶结点。最后将实例分到叶结点的类中。

- 对决策树的认识
    - 可将决策树看作**if-then规则的集合**：由决策树的根结点到叶结点的每一条路径构建一条规则；路径上内部结点的特征对应着规则的条件，而叶结点则对应着规则的结论。决策树的路径或其对应着if-then规则集合具有互斥和完备性。即每个实例都被一条路径覆盖，且只有一条路径覆盖。
    - 也可将决策树视为在**给定特征条件下类的条件概率分布**。将特征空间划分为互补相交的单元，并在每个单元定义一个类的概率分布构成一个条件概率分布。决策树的一条路径对应于划分中的一个单元。决策树所表示的条件概率分布由各个单元给定条件下类的条件概率分布组成。假设$X$为表示特征的随机变量，$Y$为表示类的随机变量，那么这个条件概率分布可表示为$P(Y|X)$。$X$取值给定划分下单元的集合，Y取值于类的集合。各叶结点上的条件概率往往偏向于某一大类，即属于某一类的概率较大。决策树分类时将该节点的实例强行分到条件概率大的那一类。

### 1.2 决策树学习

假设给定训练数据集
$$
D = \{(x_1,y_1),(x_2,y_2),...,(x_N,y_n)\}
$$
其中, $x_i=(x_i^{(1)},x_i^{(2)},...,x_i^{(n)})^T$为输入实例（特征向量），$n$为特征个数，$y_i\in \{1,2,...,K\}$为类标记，$i=1,2,...,N$，N为样本容量。决策树学习的目标是根据给定的训练数据集构建一个决策树模型，使它能够对实例进行正确的分类。

决策树学习的损失函数通常是正则化的极大似然函数，学习的策略是以损失函数为目标函数的最小化。从所有可能的决策树中选取最优决策问题是`NP hard`问题，所以现实中决策树学习算法通常采用启发式方法，近似求解这一优化问题。这样得到的决策树是次最优的(sub-optimal)。

决策树学习的算法通常是一个递归地选择最优特征，并根据该特征对训练数据进行分割，使得对各个子数据集有一个最好的分类的过程。这个过程对应着特征空间的划分，也对应着决策树的构建。

由于生成的决策树可能存在过拟合，因此需要自下而上进行剪枝，将数变得简单，使之更具泛化能力。具体地，去掉过于细分的叶结点，使其回退到父结点，甚至更高的结点，然后将父结点或更高的结点改为新的叶节点。

决策树的生成对应于模型的局部选择，剪枝对应于模型的全局选择。换而言之，决策树生成只考虑局部最优，剪枝则考虑全局最优。

## 2. 特征选择

特征选择在于选取对训练数据具有分类能力的特征，决定用哪个特征来划分特征空间，通常特征选择的准则是信息增益或信息增益比。

- 熵(entropy): 表示随机变量不确定性的度量

设$X$是一个取有限个值的随机变量，其概率分布为
$$
P(X=x_i)=p_i,i=1,2,...,n
$$
熵定义为
$$
H(x)=-\sum_{i=1}^n p_i \log p_i
$$
上式中，若$p_i=0$，则定义$0\log 0=0$；对数以2或者e为底，这时熵的单位分别称为比特(bit)或者纳特(nat)。熵只依赖于X的分布，与其取值无关，因此也可将X的熵记作$H(p)$, 即
$$
H(p)=-\sum_{i=1}^n p_i \log p_i
$$
熵越大，不确定越大

In [2]:
%matplotlib inline
from IPython import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import networkx as nx

In [3]:
def entropy(P):
    '''
    P为概率分布
    '''
    return np.dot(P, np.log2(P))

- 条件熵(conditional entropy): 表示在已知随机变量X的条件下随机变量Y的不确定性。

$$
H(Y|X)=\sum_{i=1}^n P(X=x_i)H(Y|X=x_i)
$$

其中，$H(Y|X=x_i)=-\sum_j P(Y=y_j|X=x_i)\log P(Y=y_j|X=x_i)$，表示在$X=x_i$时Y的不确定程度；$p(Y=y_j|X=x_i) = \frac{p(X=x_i, Y=y_j)}{p(X=x_i)}$。
> 如果X与Y无关，则有$H(Y|X)=H(Y)$；如果Y由X唯一决定，则有$H(Y|X)=0$

In [4]:
def conditional_entropy(P_XY):
    '''
    P_XY为X和Y的联合概率分布shape(x_size, y_z)
    '''
    return np.sum([np.sum(P_XY[i]) * entropy(P_XY[i, :]/np.sum(P_XY[i])) 
                   for i in P_XY.shape[1]])

>定义5.2 （信息增益）特征A对训练数据集D的信息增益$g(D,A)$，定义为集合D的经验熵$H(D)$与特征$A$给定条件下$D$的经验条件熵H(D|A)之差，即$g(D,A)=H(D)-H(D|A)$。
>可以理解为A对于减少D的不确定的贡献。

一般地，熵$H(Y)$与条件熵$H(Y|X)$之差称为互信息。决策树学习中的信息增益等价于训练数据集中类于特征的互信息。

决策树学习应用信息增益准则选择特征。给定训练集D合特征A，经验熵$H(D)$表示对数据集D进行分类的不确定性。而经验条件熵$H(D|A)$表示特征A在给定的条件下对数据集D进行分类的不确定性。**信息增益，就表示由于特征A而使得对数据集D的分类的不确定性减少的程度。** 信息增益大的特征具有更强的分类能力。

根据信息增益准则的特征选择方法是：对训练集D，计算其每个特征的信息增益，选择信息增益最大的特征。

假设训练集为$D, |D|$表示其样本个数。设有$K$个类$C_k, k=1,2,...,K$，$C_k$为属于类$C_k$的样本数量，$\sum_{k=1}^K|C_k|=|D|$。
设特征A有n个不同的取值$\{a_1,a_2,...,a_n\}$，根据特征A的取值将D划分为n个子集$D_1,D_2,...,D_n, |D_i|$为样本为$D_i$的样本个数，$\sum_{i=1}^n|D_i|=|D|$。
记子集$D_i$中属于类$C_k$的样本的集合为$D_{ik}$，即$D_{ik}=D_i \cap C_k$, $D_{ik}$为$D_{ik}$的样本个数

**算法5.1 （经验信息增益的算法）**
- 输入: 训练集D和特征A
- 输出：特征A对训练集D的信息增益$g(D, A)$
- 算法过程
    - 计算数据集D的经验熵$H(D)$
    $$
    H(D)=-\sum_{k=1}^K\frac{|C_k|}{|D|}\log_2\frac{|C_k|}{|D|}
    $$
    - 计算A对D的经验条件熵$H(D|A)$
    $$
    H(D|A)=\sum_{i=1}^n\frac{|D_i|}{|D|}H(D_i)=-\sum_{i=1}^n\frac{|D_i|}{|D|}\sum_{k=1}^K \frac{|D_{ik}|}{|D_i|}\log_2\frac{|D_{ik}|}{|D_i|}
    $$
    - 计算信息增益
    $$
    g(D,A)=H(D)-H(D|A)
    $$

In [5]:
def info_gain(X, y, i):
    """
    X: 特征矩阵 (m,n)
    y: 类别向量 (n,)
    i: X的第i+1列特征
    """
    Xi = X[:,i]
    prob_y = {c: np.sum(y==c)/y.size for c in np.unique(y)}  # 计算y的概率分布
    H_y = -np.sum([prob_y[p]*np.log2(prob_y[p]) for p in prob_y])  # 计算y的经验熵
    Xi_y_dict = {xi: y[Xi==xi] for xi in np.unique(Xi)}  # 找出对应xi的y
    probs = {xi: {c: np.sum(Xi_y_dict[xi]==c)/Xi_y_dict[xi].size for c in np.unique(Xi_y_dict[xi])} 
                 for xi in Xi_y_dict}  # 各xi下y的概率分布
    H_yi = {xi: -np.sum([probs[xi][p]*np.log2(probs[xi][p]) for p in probs[xi]]) 
                for xi in probs}  # 计算xi对应的y的经验熵
    H_y_x = np.sum([H_yi[xi]*(Xi_y_dict[xi].size / y.size) for xi in H_yi])  # 计算Xi对y的经验条件熵
    return H_y - H_y_x

In [6]:
X = np.array([[1, 2],
             [2, 1],
             [1, 2]])
y = np.array([1, 0, 1])
i = 0

- 信息增益比

以信息增益作为训练数据集的特征，存在偏向于选择取值较多的特征的问题。使用信息增益比(information gain ratio)可以对这一问题进行校正。这是特征选择的另一准则。

>定义5.3 （信息增益比） 特征A对D的训练增益比$\mathrm{g_R(D,A)}$定义为其信息增益$\mathrm{g_R(D,A)}$与训练数据集D关于特征A的值的熵$H_A(D)$之比，即
>$$
g_R(D,A)=\frac{g(D,A)}{H_A(D)}
$$
>其中，$H_A(D)=-\sum_{i=1}^n\frac{|D_i|}{|D|}\log_2 \frac{|D_i|}{|D|}$，n是特征A取值的个数。

In [7]:
def info_grain_ratio(X, y, i):
    ig = info_gain(X, y, i)
    Xi = X[:, i]
    prob_xi_y = {xi: len(y[Xi==xi])/len(y) for xi in np.unique(Xi)}  # 找出对应xi的y
    H_y = -np.sum([prob_xi_y[xi] * np.log2(prob_xi_y[xi]) for xi in prob_xi_y])
    return ig / H_y

## 3. 模型学习

### 3.1 `ID3`算法

ID3算法的核心是在决策树各个节点上应用信息增益准则选择特征，递归地构建决策树。

具体方法是：从根结点开始，对结点计算所有可能的特征的信息增益，选择信息增益最大的特征作为结点的特征，由该特征的不同取值建立子节点；再对子结点递归调用以上方法，构建决策树；直到所有特征的信息增益均很小或没有特征可以选择为止

**算法 5.2 （ID3算法）**
- 输入：训练集D，特征A，阈值$\epsilon$
- 输出：决策树T
- 算法过程
    - 若D中所有实例都属于同一类$C_k$, 则T为单结点树，并将类$C_k$作为该结点的类标记，返回T；
    - 若$A=\emptyset$，则$T$为单结点树，并将D中实例数最大的类$C_k$作为该结点的类标记，返回T；
    - 否则，按照算法5.1计算A中各特征对D的信息增益，选择信息增益最大的特征$A_g$;
    - 如果$A_g$的信息增益小于阈值$\epsilon$,则T为单结点树，并将D中实例数最大的类$C_k$作为该结点的类标记，返回T；
    - 否则，对$A_g$的每一可能值$a_i$，依$A_g=a_i$将D分割为若干非空子集$D_i$，将$D_i$中最大的类作为标记，构建子结点，由结点及其子结点构成树T，返回T；
    - 对第i个子结点，以$D_i$为训练集，以$A-\{A_g\}$为特征集，递归地调用以上步骤，得到子树$T_i$，返回$T_i$。

### 3.2 `C4.5`的生成算法

c4.5算法与ID3算法相似，C4.5算法对ID3算法进行了改进，在其决策树的生成过程中，使用**信息增益比**来选择特征。

**算法5.3 （C4.5的生成算法）**

- 输入：训练集D，特征A，阈值$\epsilon$
- 输出：决策树T
- 算法过程
    - 若D中所有实例都属于同一类$C_k$, 则T为单结点树，并将类$C_k$作为该结点的类标记，返回T；
    - 若$A=\emptyset$，则$T$为单结点树，并将D中实例数最大的类$C_k$作为该结点的类标记，返回T；
    - 否则，按照算法5.1计算A中各特征对D的信息增益比，选择信息增益比最大的特征$A_g$;
    - 如果$A_g$的信息增益小于阈值$\epsilon$,则T为单结点树，并将D中实例数最大的类$C_k$作为该结点的类标记，返回T；
    - 否则，对$A_g$的每一可能值$a_i$，依$A_g=a_i$将D分割为若干非空子集$D_i$，将$D_i$中最大的类作为标记，构建子结点，由结点及其子结点构成树T，返回T；
    - 对第i个子结点，以$D_i$为训练集，以$A-\{A_g\}$为特征集，递归地调用以上步骤，得到子树$T_i$，返回$T_i$。

## 4. 决策树的剪枝

决策树生成算法递归地生成决策树，直到不能继续下去为止，这样产生的数往往会出现过拟合现象。原因在于学习时过多地考虑如何提高训练数据的分类准确性，从而构建了过于复杂的决策树。解决的办法是简化已生成的决策树，即执行**剪枝(pruning)**操作。具体地，从已生成的树上裁减一些子树或叶结点，并将其根结点或父结点作为新的叶结点。

决策树的剪枝往往通过极小化决策树整体的**损失函数(loss function)**或**代价函数(cost function)**来实现。设树T的叶节点个数为$|T|$，t是树T的叶结点，该叶结有$N_t$个样本点，其中k类的样本点有$N_{tk}$个，$k=1,2,...,K$个，$H_t(T)$为叶结点t上的经验熵，$\alpha\geq 0$为参数，则决策树学习的损失函数可以定义为
$$
C_{\alpha}(T)=\sum_{t=1}^{|T|}N_tH_t(T)+\alpha |T|
$$
其中经验熵为
$$
H_t(T)=-\sum_k\frac{N_{tk}}{N_t}\log\frac{N_{tk}}{N_t}
$$

在损失函数中，将右端的第1项记作
$$
C(T)=\sum_{t=1}^{|T|}N_tH_t(T)=-\sum_{t=1}^{|T|}\sum_{k=1}^K N_{tk}\log\frac{N_{tk}}{N_t}
$$

这时有
$$
C_{\alpha}(T)=C(T)+\alpha |T|
$$

其中$C(T)$表示模型对训练数据的预测误差，即模型与训练数据的拟合程度，$|T|$表示模型复杂度，参数$\alpha\geq 0$控制两者之间的影响。$\alpha=0$意味着只考虑模型与训练数据的拟合程度，不考虑模型的复杂度; $\alpha$较大促使选择简单的模型。

以上定义的损失函数的极小化等价于正则化的极大似然估计。所以，利用损失函数最小化原则进行剪枝就是用正则化的极大似然估计进行模型选择。

剪枝，就是当$\alpha$确定时，选择损失函数最小的模型，即损失函数最小的子树。决策树生成学习局部的模型，而决策树剪枝则学习整体的模型。

**算法5.4 （树的剪枝算法）**

- 输入：生成算法产生的整个树T，参数$\alpha$
- 输出：修剪后的子树$T_{\alpha}$
- 算法过程
    - 计算每个节点的经验熵。
    - 递归地从树的叶节点向上收缩。设一组兄弟叶结点(即有同一父结点)回缩到其父结点之前与之后的整体树分别为$T_B$与$T_A$，其对应的损失函数值分别为$C_{\alpha}(T_B)$与$C_{\alpha}(T_A)$，如果$C_{\alpha}(T_B)\leq C_{\alpha}(T_A)$，则进行剪枝，即将父结点变为新的叶结点。
    - 返回(2)，直至不能继续为止，得到损失函数最小的子树$T_{\alpha}$。
    
由于$C_{\alpha}(T_B)\leq C_{\alpha}(T_A)$只需考虑两个树的损失函数的差，其计算可以在局部进行。

>除以上基于正则化的剪枝方法外，也有采用留出法评估决策树的泛化能力，即从预留一部分样本数据用作“验证集”以进行泛化性能评估(`周志华, 2017`)。该策略下的剪枝才做也分为**“预剪枝”(prepruning)**和**“后剪枝”(postpruning)**(`Quinlan, 1993`)。
>- 预剪枝是指在决策树生成过程中，对每个结点在划分前进行估计，若当前结点的划分不能带来决策树泛化性能提升，则停止划分并将当前结点标记为叶节点。
>- 后剪枝是指先从训练集生成一颗完整的决策树，然后自底向上地对非叶节点进行考察，若将该结点对应的子树替代为叶节点能带来决策树泛化性能提升，则将该子树替换为叶节点。

In [ ]:
from copy import deepcopy

class DecisionTreeClassifier:
    '''
    算法实现思想：树的生成过程为结点的新增及其标记(name)过程，因此需判断何时新增结点？新增的节点是否满足叶结点的条件？
        新增的节点应选择哪个属性作为其标记？
    '''
    def __init__(self, feature, label, epsilon, alpha, mode='id.3'):
        if type(feature) != pd.core.frame.DataFrame or type(label) != pd.core.frame.DataFrame:
            raise(TypeError, '请使用pandas.DataFrame组织数据!')
        else:
            self.feature = feature
            self.label = label
            self.alpha = alpha
            self.epsilon = epsilon
            self.tree = nx.DiGraph()
            self.node_id = 0
            self.no_name_nodes = [0]  # 尚未分类的节点，动态变化
            self.mode = mode
            self.tree.add_node(0)
            self.tree.nodes[0]["X"] = self.feature
            self.tree.nodes[0]["y"] = self.label
        
    def info_gain(self, X, y, col_name)
        Xi = X[col_name].values
        y = y['label'].values
        prob_y = {c: np.sum(y==c)/y.size for c in np.unique(y)}  # 计算y的概率分布
        H_y = -np.sum([prob_y[p]*np.log2(prob_y[p]) for p in prob_y])  # 计算y的经验熵
        Xi_y_dict = {xi: y[Xi==xi] for xi in np.unique(Xi)}  # 找出对应xi的y
        probs = {xi: {c: np.sum(Xi_y_dict[xi]==c)/Xi_y_dict[xi].size for c in np.unique(Xi_y_dict[xi])} 
                     for xi in Xi_y_dict}  # 各xi下y的概率分布
        H_yi = {xi: -np.sum([probs[xi][p]*np.log2(probs[xi][p]) for p in probs[xi]]) 
                    for xi in probs}  # 计算xi对应的y的经验熵
        H_y_x = np.sum([H_yi[xi]*(Xi_y_dict[xi].size / y.size) for xi in H_yi])  # 计算Xi对y的经验条件熵
        return H_y - H_y_x
    
    def info_grain_ratio(self, X, y, col_name):
        Xi = X[col_name].values
        y = y['label'].values
        ig = info_gain(X, y, col_name)
        prob_xi_y = {xi: len(y[Xi==xi])/len(y) for xi in np.unique(Xi)}  # 找出对应xi的y
        H_y = -np.sum([prob_xi_y[xi] * np.log2(prob_xi_y[xi]) for xi in prob_xi_y])
        return ig / H_y
    
    def gini_index(self, X, y)
    
    def generate_successors(self, node):
        '''
        从属性列表中选出目标属性，并和当前备用节点node建立联系
        node的属性: 
            name -- 续分属性attribute或者leaf_node
            category -- 对应样本的类别
            X -- 特征向量
            y -- 类别
        '''
        X = self.tree.nodes[node]["X"]
        y = self.tree.nodes[node]["y"]
        new_add_nodes = []
        frequency_y = [(v, np.sum(y==v)) for v in np.unique(y.label.values)]
        max_y = max(frequency_y, key=lambda x: x[1])[0]
        # case 1: 如果无续分属性或样本同属一类, 当前节点置为叶结点
        if X.size == 0 or len(frequency_y) == 1:
            self.tree.nodes[node]["name"] = 'leaf_node'
            self.tree.nodes[node]["category"] = max_y
        else:  # 存在续分属性
            attr_set = set(X.columns.values)  # 节点对应的属性
            if self.mode == "id.3":
                info_gain_list = [(attr, self.info_gain(X, y, attr)) 
                                  for attr in attr_set]  # 计算各属性的信息增益
            elif self.mode = "c4.5":
                info_gain_list = [(attr, self.info_gain_ratio(X, y, attr)) 
                                  for attr in attr_set]  # 计算各属性的信息增益比
            else:
                raise(ValueError, "请选择正确的分类准则('id.3'或'c4.5')")

            target_attr, target_info_gain = max(info_gain_list, lambda x: x[1])  # 选择信息增益(比)最大的属性
            # case 2: 如果信息增益小于阈值或者target_attr所有值相同，则不继续生成结点，当前节点重置为叶结点
            if len(np.unique(X[target_attr].values)) == 1 or target_info_gain < self.epsilon:
                self.tree.nodes[node]["name"] = 'leaf_node'
                self.tree.nodes[node]["category"] = max_y
            else:  # case 3: 继续往下增加结点, 当前节点名称置为续分属性名
                self.tree.nodes[node]["name"] = target_attr  # 续分属性名
                Xi = X[target_attr]
                for xi in np.unique(Xi.values):
                    self.node_id += 1
                    self.tree.add_edge(node, self.node_id, value=xi)  # 添加连边
                    self.tree.nodes[self.node_id]["X"] = X[Xi == xi].drop(target_attr)  # 获取目标值，且删除该属性
                    self.tree.nodes[self.node_id]["y"] = y.iloc[X[Xi == xi].index]
                    new_add_nodes.append(self.node_id)

        return new_add_nodes
            
    def generate_tree(self):
        while self.no_name_nodes:
            new_nodes = []
            for node in self.no_name_nodes:
                new_add_nodes = self.generate_successors(node)
                new_nodes.extend(new_add_nodes)

            self.no_name_nodes = new_nodes
                
    def loss(self, tree):
        CT_list = []
        for i in tree:
            if tree.nodes[i]['category']=='leaf_node':  # 叶结点
                y = tree.nodes[i]['y']
                frequency = np.array([y[y.label == c].size for c in np.unique(y.label.values)])
                CT_list.append(np.dot(frequency, np.log2(frequency/y.size)))
                
        return -np.sum(CT_list) + self.alpha*len(CT_list)
    
    def regularization(self):
        """
        利用正则化剪枝
        """
        
            
    def prepruning(self):
        """
        预剪枝操作
        """
        
        
    def postpruning(self):
        """
        后剪枝操作
        """
        tree = deepcopy(self.tree)  # 深度拷贝训练好的决策树
        node_sets = [i for i in self.tree if self.tree.nodes[i]['category']=='leaf_node']  # 初始化需遍历的结点
        while node_sets:
            
            
    def predict(self, x_dict):
        """
        x_series: 可以为字典
        """
        node = 0
        while True:
            xi = self.tree.nodes[node]["name"]
            for i in self.tree.successors(node):
                if self.tree.edges[node][i]['value'] == xi:
                    break
            else:  # 如果没有相等的值
                print(f'目标属性值不存在{self.tree.nodes[node]["name"]} = {xi}')
                break
                
            if self.tree.nodes[i]['name'] == 'leaf_node':
                return self.tree.nodes[i]['category']
            else:
                node = i

## 5. `CART`算法

分类与回归数(classification and regression tree, CART)模型由`Breiman`等(1984)提出，是应用广泛的决策树学习方法。`CART`同样由特征选择、树的生成及剪枝组成，既可用于分类也可以用于回归。

CART是在给定输入随机变量X条件下输出随机变量Y的条件概率分布的学习方法。CART假定决策树是二叉树，内部结点特征的取值为“是”和“否”，左分支是取值为“是”的分支，有分支是取值为“否”的分支。这样的决策树等价于递归地二分每个特征，将输入控件即特征空间划分为有限个单元，并在这些单元上确定预测的概率分布，也就是输入给定的条件下输出的条件概率分布。

CART算法由以下两步组成：
- 决策树生成：基于训练数据集生成决策树，生成的决策树要尽量大；
- 决策树剪枝：用验证集对已生成的树进行剪枝并选择最优子树，这时用损失函数最小作为剪枝的标准。

### 5.1 `CART`生成

对回归树用平方误差最小化准则，对分类树用基尼指数最小化准则，进行特征选择，生成二叉树。

#### 回归树的生成
假设X与Y分别为输入和输出变量，并且Y是连续变量，给定训练数据集
$$
D=\{(x_1,y_1),(x_2,y_2),...,(x_N,y_N)\}
$$
考虑如何生成回归树。

一颗回归树对应着输入控件的一个划分以及在划分的单元上的输出值。假设已将输入控件划分为M各单元$R_1,R_2,...R_M$，并且在每个单元$R_m$上有一个固定的输出值$c_m$，于是回归树模型可表示为
$$
f(x)=\sum_{m=1}^M c_m I(x\in R_m)
$$

当输入空间的划分确定时，可以用平方误差$\sum_{x_i\in R_m}(y_i-f(x_i))^2$来表示回归树对于训练数据的预测误差，用平方误差最小的准则求解每个单元上的最优输出值。显然，$c_m$的最优值为
$$
\hat{c}_m=\mathrm{average}(y_i|x_i\in R_m)
$$

问题时怎样对输入控件进行划分。可以使用启发式的方法，选择第j个变量$x_{(j)}$和它的取值s，作为切分变量和切分点，并定义两个区域:
$$
R_1(j,s)=\{x|x^{(j)}\leq s\} \text{ 和 } R_2(j,s)=\{x|x^{(j)}> s\}
$$
然后寻找最优切分变量j和最优切分点s。具体地，求解
$$
\min_{j,s}\left[\min_{c_1}\sum_{x_i\in R_1(j,s)}(y_i-c_1)^2+\min_{c_2}\sum_{x_i\in R_2(j,s)}(y_i-c_2)^2\right]
$$
对于固定输入变量j可以找到最优切分点s。
$$
\hat{c}_1=\mathrm{average}(y_i|x_i\in R_1(j,s)) \text{ 和 }  \hat{c}_2=\mathrm{average}(y_i|x_i\in R_2(j,s))
$$
遍历所有输入变量，找到最优的切分变量j构成一对$(j,s)$，将输入空间划分为两个区域，然后对每个区域重复以上划分过程，知道满足停止条件为止。

这样，可以生成一颗回归树。这样的回归树通常称为**最小二乘回归树(`least squares regression tree`)**。

**算法5.5 （最小二乘回归树生成算法）**

In [14]:
t = nx.DiGraph()

In [28]:
t.nodes[0]['type'] = 'root'

In [26]:
for i in t.nodes():
    print(i)

0


In [29]:
a = np.arange(10)
b = a[5]
b = -1

In [32]:
t.add_edges_from([(0,1), (1, 2), (2, 3), (2, 4), (0, 4)])

In [33]:
nx.shortest_path(t, source=0, target=4)

[0, 4]

In [35]:
df = pd.DataFrame({'col1': [1,2,3], 'col2': [2,3,4], 'col3': [1,1,1]})

In [38]:
set(df.columns.values)

{'col1', 'col2', 'col3'}

In [39]:
type(df)

pandas.core.frame.DataFrame

In [48]:
df.iloc[df[df.col1 == 2].index]

,col1,col2,col3
1,2,3,1


In [49]:
df

,col1,col2,col3
0,1,2,1
1,2,3,1
2,3,4,1


In [61]:
dfa = dfa.drop('col2', axis=1)

In [74]:
dfa.col1.size

3

In [64]:
x = dfa.drop('col1', axis=1)

In [68]:
len(x.columns)

0

In [71]:
if x.size:
    print(1)

In [72]:
dfa.size

3